In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json
import math
import decoupler as dc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from sklearn.metrics import roc_curve, auc

In [9]:
adata_file = "data/TianKampmann2021_CRISPRi.h5ad"
adata = sc.read_h5ad(adata_file)
adata

AnnData object with n_obs × n_vars = 32300 × 33538
    obs: 'guide_id', 'perturbation', 'tissue_type', 'celltype', 'cancer', 'disease', 'perturbation_type', 'organism', 'batch', 'nperts', 'ngenes', 'ncounts', 'percent_mito', 'percent_ribo', 'percent_hemo'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [10]:
adata.obs

,guide_id,perturbation,tissue_type,celltype,cancer,disease,perturbation_type,organism,batch,nperts,ngenes,ncounts,percent_mito,percent_ribo,percent_hemo
AAACCCAAGGGTTGCA,CREBBP_i7,CREBBP,primary,iPSC-induced neuron,False,healthy,CRISPR,human,1,1,745,1013.0,7.897335,5.824284,0.0
AAACCCAAGGTAGCCA,SH3RF1_i2,SH3RF1,primary,iPSC-induced neuron,False,healthy,CRISPR,human,1,1,6411,28205.0,7.147669,8.236128,0.0
AAACCCACAAACTAGA,TAF1_i5,TAF1,primary,iPSC-induced neuron,False,healthy,CRISPR,human,1,1,4739,15249.0,7.934947,8.302184,0.0
AAACCCACAGAATTCC,UBTD2_i1,UBTD2,primary,iPSC-induced neuron,False,healthy,CRISPR,human,1,1,4665,14084.0,5.467197,7.078955,0.0
AAACCCAGTAGCGTTT,FRMD4A_i2,FRMD4A,primary,iPSC-induced neuron,False,healthy,CRISPR,human,1,1,4756,15374.0,8.228177,6.524002,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGCAGGTCTCG,FAM57B_i2,FAM57B,primary,iPSC-induced neuron,False,healthy,CRISPR,human,4,1,4398,12748.0,7.232507,8.024788,0.0
TTTGTTGCAGTTGCGC,XRCC1_i1,XRCC1,primary,iPSC-induced neuron,False,healthy,CRISPR,human,4,1,3080,7539.0,5.239422,8.117788,0.0
TTTGTTGGTCCCGGTA,RPL14_i2,RPL14,primary,iPSC-induced neuron,False,healthy,CRISPR,human,4,1,4941,15206.0,6.096278,6.484283,0.0
TTTGTTGGTTGATGTC,PMPCA_i2,PMPCA,primary,iPSC-induced neuron,False,healthy,CRISPR,human,4,1,3023,7189.0,4.061761,8.540826,0.0


In [6]:
print(adata.var)

                  ensembl_id  ncounts  ncells
MIR1302-2HG  ENSG00000243485      0.0       0
FAM138A      ENSG00000237613      0.0       0
OR4F5        ENSG00000186092      4.0       4
AL627309.1   ENSG00000238009    307.0     304
AL627309.3   ENSG00000239945     33.0      33
...                      ...      ...     ...
AC233755.2   ENSG00000277856      0.0       0
AC233755.1   ENSG00000275063      0.0       0
AC240274.1   ENSG00000271254   1971.0    1859
AC213203.1   ENSG00000277475      0.0       0
FAM231C      ENSG00000268674      1.0       1

[33538 rows x 3 columns]


In [8]:
adata.obs.describe()

,nperts,ngenes,ncounts,percent_mito,percent_ribo,percent_hemo
count,32300.000000,32300.000000,32300.000000,32300.000000,32300.000000,32300.000000
mean,0.986471,4431.089443,14820.948266,7.188241,7.240245,0.000011
std,0.115528,1473.834862,8176.411131,4.047170,1.769687,0.000655
min,0.000000,389.000000,506.000000,0.366300,1.087596,0.000000
25%,1.000000,3752.000000,9815.500000,5.375521,6.334525,0.000000
50%,1.000000,4521.000000,13609.000000,6.382979,7.212476,0.000000
75%,1.000000,5351.000000,18831.250000,7.622260,8.140817,0.000000
max,1.000000,9261.000000,82442.000000,60.136742,28.703704,0.110375


In [10]:
# identify controls (heuristic: explicit "control" in perturbation or guide names or nperts==0)
ctrl_mask = (adata.obs['perturbation'].str.contains('control', case=False, na=False)) | \
            (adata.obs['guide_id'].str.contains('non-target', case=False, na=False)) | \
            (adata.obs['nperts'] == 0)
adata.obs['is_control'] = ctrl_mask

# create a per-cell label: target gene name or "control"
adata.obs['ptb_label'] = adata.obs['perturbation'].astype(str).replace('nan', 'unknown')
adata.obs.loc[adata.obs['is_control'], 'ptb_label'] = 'control'

# Option: simplify labels to only TFs you care about
# tf_list = [...]  # your TF list
# adata.obs['ptb_label'] = adata.obs['ptb_label'].where(adata.obs['ptb_label'].isin(tf_list), other='other')

# Save labels for evaluation
labels = adata.obs['ptb_label']


In [44]:
print("labels.shape: ", labels.shape)
labels

labels.shape:  (32300,)


AAACCCAAGGGTTGCA    CREBBP
AAACCCAAGGTAGCCA    SH3RF1
AAACCCACAAACTAGA      TAF1
AAACCCACAGAATTCC     UBTD2
AAACCCAGTAGCGTTT    FRMD4A
                     ...  
TTTGTTGCAGGTCTCG    FAM57B
TTTGTTGCAGTTGCGC     XRCC1
TTTGTTGGTCCCGGTA     RPL14
TTTGTTGGTTGATGTC     PMPCA
TTTGTTGTCTTACTGT       HTT
Name: ptb_label, Length: 32300, dtype: object

## Run KALE on multiple datasets

In [3]:
adata_files = [
    "data/TianKampmann2021_CRISPRi.h5ad",
    "data/TianKampmann2021_CRISPRa.h5ad",
    "data/ReplogleWeissman2022_rpe1.h5ad",
    "data/NormanWeissman2019_filtered.h5ad",
    "data/AdamsonWeissman2016_GSM2406681_10X010.h5ad",
    "data/FrangiehIzar2021_RNA.h5ad"
]

for adata_file in adata_files:
    print("Running KALE for test case: ", adata_file)

    file_name = adata_file.split('/')[-1].replace('.h5ad', '')

    # run script
    print("Running KALE for test case without weights without ignore_zeros")
    !uv run src/kale.py --gene_exp_file {adata_file} --prior_file data/causal_priors.tsv --output_file data/_kale_scores_unweighted_{file_name}.tsv --pvalue_output_file data/_kale_pvalues_unweighted_{file_name}.tsv --ignore_zeros False --cores 8 --method rank_of_ranks --min_targets 1 --weighted False

    print("Running KALE for test case without weights ignoring zeros")
    !uv run src/kale.py --gene_exp_file {adata_file} --prior_file data/causal_priors.tsv --output_file data/_kale_scores_unweighted_ignore_zeros_{file_name}.tsv --pvalue_output_file data/_kale_pvalues_unweighted_ignore_zeros_{file_name}.tsv --ignore_zeros True --cores 8 --method rank_of_ranks --min_targets 1 --weighted False

    print("Running KALE for test case with weights without ignore_zeros")
    !uv run src/kale.py --gene_exp_file {adata_file} --prior_file data/causal_priors.tsv --output_file data/_kale_scores_weighted_{file_name}.tsv --pvalue_output_file data/_kale_pvalues_weighted_{file_name}.tsv --ignore_zeros False --cores 8 --method rank_of_ranks --min_targets 1 --weighted True --weighted_power_factor 1

    print("Running KALE for test case with weights ignoring zeros")
    !uv run src/kale.py --gene_exp_file {adata_file} --prior_file data/causal_priors.tsv --output_file data/_kale_scores_weighted_ignore_zeros_{file_name}.tsv --pvalue_output_file data/_kale_pvalues_weighted_ignore_zeros_{file_name}.tsv --ignore_zeros True --cores 8 --method rank_of_ranks --min_targets 1 --weighted True --weighted_power_factor 1

Running KALE for test case:  data/TianKampmann2021_CRISPRi.h5ad
TianKampmann2021_CRISPRi
Running KALE for test case:  data/TianKampmann2021_CRISPRa.h5ad
TianKampmann2021_CRISPRa
Running KALE for test case:  data/ReplogleWeissman2022_rpe1.h5ad
ReplogleWeissman2022_rpe1
Running KALE for test case:  data/NormanWeissman2019_filtered.h5ad
NormanWeissman2019_filtered
Running KALE for test case:  data/AdamsonWeissman2016_GSM2406681_10X010.h5ad
AdamsonWeissman2016_GSM2406681_10X010
Running KALE for test case:  data/FrangiehIzar2021_RNA.h5ad
FrangiehIzar2021_RNA


## Run Viper, MLM, ULM, 

In [3]:
net_file = "data/causal_priors.tsv"
effect_map = {"upregulates-expression": 1, "downregulates-expression": -1}
net = pd.read_csv(
    net_file,
    sep="\t",
    names=["source", "weight", "target"],
    usecols=[0, 1, 2],
    converters={"weight": effect_map.get}
)[["source", "target", "weight"]]


# Run Decoupler Methods
methods_to_run = ["viper", "mlm", "ulm"]


for adata_file in adata_files:
    print(f"Processing {adata_file}...")
    adata = sc.read_h5ad(adata_file)
    dc.mt.decouple(adata, net, tmin=1, methods=methods_to_run)

    for method in methods_to_run:
        method_scores = adata.obsm[f"score_{method}"]
        method_scores_df = pd.DataFrame(
            method_scores,
            index=adata.obs_names,
            columns=[col for col in adata.var_names if col in net['source'].values]
        )
        method_scores_df.to_csv(f"data/_{method}_scores_{adata_file.split('/')[-1].replace('.h5ad', '')}.tsv", sep="\t")
        print(f"Saved {method} scores to data/{method}_scores_{adata_file.split('/')[-1].replace('.h5ad', '')}.tsv")

Processing data/TianKampmann2021_CRISPRi.h5ad...
Saved viper scores to data/viper_scores_TianKampmann2021_CRISPRi.tsv
